In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive



/var/folders/bx/n7x8tlw17p9_m_8d4g08505w0000gn/T/ipykernel_6190/2758595007.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/fluffysnail/Library/CloudStorage/OneDrive-Personal/dtu/current/42186_modelml/42186_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def model3(obs, ids1, ids2):
    hyper_sigma = 1/8
    coach_skills = pyro.sample("coach_skill", dist.Normal(torch.zeros(2), hyper_sigma).to_event(1))
    
    with pyro.plate('matches', obs.shape[0]):   

        coach1_perf = pyro.sample('coach1_perf', dist.Normal(coach_skills[ids1], hyper_sigma))
        coach2_perf = pyro.sample('coach2_perf', dist.Normal(coach_skills[ids2], hyper_sigma))

        perf_diff = coach1_perf - coach2_perf
    
        y = pyro.sample("y_coach1_win", dist.Normal(perf_diff, hyper_sigma), obs=obs)

    return y


In [3]:
optimizer = Adam({"lr": 0.01})
guide = AutoDiagonalNormal(model3)
svi = SVI(model3, guide, optimizer, loss=Trace_ELBO())

# Training loop
num_steps = 5000
obs = torch.tensor([1,1,1,1,1,1,1,1,1,1,1,1])
id1 = torch.zeros(len(obs)).long()
id2 = torch.ones(len(obs)).long()
for step in range(num_steps):
    loss = svi.step(obs, id1, id2)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")




Step 0 : loss = 507.1372079849243
Step 100 : loss = 7.543281555175781
Step 200 : loss = 12.353198051452637
Step 300 : loss = 14.673108100891113
Step 400 : loss = 12.311419010162354
Step 500 : loss = 13.76845121383667
Step 600 : loss = 17.945531368255615
Step 700 : loss = 8.759594440460205
Step 800 : loss = 11.16513729095459
Step 900 : loss = 9.147468566894531
Step 1000 : loss = 13.45939016342163
Step 1100 : loss = 11.794519424438477
Step 1200 : loss = 9.444076538085938
Step 1300 : loss = 10.51755142211914
Step 1400 : loss = 13.441921710968018
Step 1500 : loss = 10.61579704284668
Step 1600 : loss = 14.916497707366943
Step 1700 : loss = 10.16316032409668
Step 1800 : loss = 12.095477104187012
Step 1900 : loss = 15.365684986114502
Step 2000 : loss = 12.894238471984863
Step 2100 : loss = 10.759790420532227
Step 2200 : loss = 15.338965892791748
Step 2300 : loss = 10.412757396697998
Step 2400 : loss = 12.399275779724121
Step 2500 : loss = 11.376036167144775
Step 2600 : loss = 10.9284181594848

In [4]:
from pyro.infer import Predictive

predictive = Predictive(model3, guide=guide, num_samples=2000,
                        return_sites=("coach_skill", "coach_skill"))

samples = predictive(obs,id1, id2)


ValueError: Shape mismatch inside plate('matches') at site coach1_perf dim -1, 12 vs 2
   Trace Shapes:       
    Param Sites:       
   Sample Sites:       
coach_skill dist    | 2
           value  1 | 2
    matches dist    |  
           value 12 |  

In [ ]:
import seaborn as sns

coach1_skill = samples["coach_skill"].detach().squeeze()

sns.histplot(coach1_skill, kde=True, alpha=0.2)
plt.legend("coach_skill")
plt.show()